# PART 5 - Programme d'identification de faux billets

# Fichier à tester

In [29]:
csv = 'Sources/example.csv'

# Écriture du programme

In [32]:
import pandas as pd
import pickle

pd.set_option('mode.chained_assignment', None)

def detecteur_billet(csv):
    """ Détecte si un billet est vrai ou non. Retourne un dataframe
    Args:
        csv (CSV file): Fichier CSV avec un billet par ligne.
    """
    
    # Lecture du fichier CSV
    df = pd.read_csv(csv)
    data = df.iloc[:,:-1].copy()
    
    # Récupération du modèle
    with open('data/model.pkl', 'rb') as f:
        my_unpickler = pickle.Unpickler(f)
        model = my_unpickler.load()
        
    # Préparation des données
    data = data[['length', 'margin_low']]
    
    # Modélisation et enregistrement des résultats ligne par ligne
    df['is_genuine'] = 2
    df['Proba'] = 0
    for i in range(0,5):
        results = model.predict([[data['length'][i],data['margin_low'][i]]])
        probabs = model.predict_proba([data.iloc[i]])
        df['is_genuine'].iloc[i] = results
        df['Proba'].iloc[i] = probabs[:,-1]

    df['Prédiction'] = df['is_genuine'].apply(lambda x: 'Vrai billet' if x == 1 else 'Faux billet')
    df['Probalité d\'être vrai'] = round(df['Proba'],3)
    
    # Renvoi l'identifiant du billet et le résultat lisible
    return df[['id', 'Prédiction','Probalité d\'être vrai']]

# Lancement du programme

In [33]:
detecteur_billet(csv)

,id,Prédiction,Probalité d'être vrai
0,A_1,Faux billet,0.013
1,A_2,Faux billet,0.011
2,A_3,Faux billet,0.034
3,A_4,Vrai billet,0.943
4,A_5,Vrai billet,0.993
